* Pure python implementation of decision trees

* Decision Tree is a binary tree that recursively splits a dataset until it is left with pure leaf nodes.
* Its a greedy algorithm

$$
H(S) = - \sum_{i=1}^{n} p(x_i) \log_2 p(x_i)
$$

$$
IG(T, A) = H(T) - \sum_{v \in \text{Values}(A)} \frac{|T_v|}{|T|} H(T_v)
$$

In [1]:
# 1. Select a feature
# 2. Select feature value randomly from range
# 3. Calculate entropy (purity of sample)
#    a. Proportion of +(ve) examples -> p =  count(feature) groupby target filter target == 1 / sum(feature)
#    b. Proportion of -(ve) examples -> n =  1 - p
#    c. entropy = -plog2(p) - nlog2(n)
#    d. entropy (general) = sigma(-Pnlog2(Pn))
# 4. Split feature according to entropy and do 3. for each split (binary tree)

### Doubts

#### What are the stopping conditions for splitting

* Maximum Depth Reached:

    * The tree is allowed to grow only up to a predefined maximum depth. Once this limit is reached, the node is not split further.
* Minimum Samples for a Split:

    * A node is split only if it contains more than a certain number of samples. If the number of samples in a node falls below this threshold, the split will not occur.
* Minimum Samples in a Leaf Node:

    * After splitting, the resulting child nodes must have at least a minimum number of samples. If this condition is violated, no split occurs.
* Impurity Improvement Below a Threshold:

    * Splitting is stopped when the improvement in the chosen impurity metric (like Gini Impurity or Entropy) is less than a predefined threshold. If the split does not significantly improve the purity of the node, it is not performed.
* All Features Used:

    * If all features have been used up or if no further features provide better splits, the process stops.
* Pure Nodes:

    * If all the samples in a node belong to the same class, the node is considered "pure," and further splitting is unnecessary.


In [2]:
import math
from sklearn.datasets import load_iris
import pandas as pd

In [3]:
# Load the Iris dataset
iris = load_iris()

# Convert to a pandas DataFrame for easier analysis
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target

# Display the first few rows
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


### Decision Tree Classifier

### Akhil's version

In [4]:
class Node():
    
    def __init__(self,X: pd.DataFrame = None, y: pd.Series = None ,\
                 feature: str = None, threshold: float = None , \
                 left: 'Node' = None , right: 'Node' = None, \
                 entropy_node: float = None, entropy_left_child: float = None ,\
                 length_node: int = None, length_left_child: int = None, length_right_child: int = None,\
                 w_left: float = None, w_right: float = None,\
                 entropy_right_child: float = None, entropy_children: float = None,path: str = None, \
                 max_info_gain: float = -math.inf, value: float = None, current_depth: int = 0):
        
        # Basic Node Params
        self.X = X
        self.y = y
        self.feature = feature      
        self.threshold = threshold  
        self.left = left            
        self.right = right
        self.current_depth = current_depth
        self.path = path
            
        # Information Gain Params
        self.entropy_node = entropy_node
        self.entropy_left_child = entropy_left_child
        self.entropy_right_child = entropy_right_child
        self.entropy_children = entropy_children
        self.length_node = length_node
        self.length_left_child = length_left_child
        self.length_right_child = length_right_child
        self.w_left = w_left
        self.w_right = w_right
        #self.info_gain = info_gain
        self.max_info_gain = max_info_gain
        
        # Leaf Params
        self.value = value
        
        # Print Param
        self.indent = "----"
        
        
    def __str__(self):
        
        indent = self.indent * self.current_depth
        
        if self.current_depth == 0:
            rep = f"|{indent}|\U0001F914{self.feature} <= {self.threshold}    \U00002B06{round(self.max_info_gain,2)}"
        
        else:
            if self.value is not None:
                rep = f"|{indent}|\U0001F343{self.path}: {self.value}"
                
            else:
                rep = f"|{indent}|\U0001F914{self.path}:{self.feature} <= {self.threshold}    \U00002B06{round(self.max_info_gain,2)}" 
        
        return rep
            


In [5]:
class DecisionTreeClassifier():
    
    
    def __init__(self, min_samples_split = 3, max_depth = 15, min_info_gain = .001):

        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.min_info_gain = min_info_gain
        
        self.root = None
        
    
    def build_tree(self, node):

        
        if len(node.y) >= self.min_samples_split and node.current_depth <= self.max_depth:
            
            left_X, left_y, right_X, right_y = self.get_best_split(node)
            if node.max_info_gain > self.min_info_gain :
                node.left = Node(left_X, left_y, current_depth = node.current_depth + 1, path = 'Left')
                node.right = Node(right_X, right_y, current_depth = node.current_depth + 1,path = 'Right')
                self.build_tree(node.left)
                self.build_tree(node.right)
            else:
                node.value = node.y.mode()[0]
            
        else:
        
            node.value = node.y.mode()[0]

            
         
            
    def get_best_split(self, node):

        X, y = node.X, node.y

        #node.info_gain = -math.inf
        
        for feature in X.columns:
            unique_values = X[feature].unique()
            for threshold in unique_values:
                # Split 
                left_y,right_y = y[X[feature] <= threshold], y[X[feature] > threshold]
                
                # Implementing a check for null left or right
                if len(left_y) > 0 and len(right_y) > 0: # Don't consider edge of X.feature
                    
                    entropy_left, entropy_right, length_node, length_left, \
                    length_right, w_left, w_right, entropy_children, \
                    gain = self.info_gain(left_y,right_y, node)

                    if gain > node.max_info_gain:
                        node.max_info_gain = gain
                        node.feature = feature
                        node.threshold = threshold
                        node.entropy_left_child = entropy_left
                        node.entropy_right_child = entropy_right
                        node.length_node = length_node
                        node.length_left_child = length_left
                        node.length_right_child = length_right
                        node.w_left = w_left
                        node.w_right = w_right
                        node.entropy_children = entropy_children
     
        

        left_X = X[X[node.feature] <= node.threshold]
        left_y = y[X[node.feature] <= node.threshold]
        right_X = X[X[node.feature] > node.threshold]
        right_y = y[X[node.feature] > node.threshold]
        

        return left_X, left_y, right_X, right_y
    
                
    def info_gain(self, left_y:pd.Series, right_y:pd.Series, node: 'Node') -> float:

        node.entropy_node = self.entropy(node.y)
        entropy_left = self.entropy(left_y)
        entropy_right = self.entropy(right_y)
        length_node = len(node.y)
        length_left = len(left_y)
        length_right = len(right_y)
        w_left = length_left/length_node
        w_right = length_right/length_node
        entropy_children = w_left * entropy_left + w_right * entropy_right
        gain = node.entropy_node - entropy_children

        return entropy_left, entropy_right, length_node, length_left, \
                    length_right, w_left, w_right, entropy_children, \
                    gain
    
    
    def entropy(self, y:pd.Series) -> float:
    
        prob = y.value_counts(normalize = True).reset_index()

        return sum([-p * math.log2(p) for p in prob.proportion])
    
    
    def fit(self, X, y):
        
        self.root = Node(X, y, path = 'root')
        #print(self.root.max_info_gain)
        self.build_tree(self.root)
        

    def print_tree(self, node = None):
        
        if node == None:
            node = self.root
            
        if node.value is not None:
            print(f"{node.current_depth - 1}->{node.path}->{node.current_depth}:`Leaf`[[{node.value}]]")
            
        else:
            if node.current_depth == 0:
                print(f"{node.current_depth}->{node.path}->{node.current_depth}:`Decision`({node.feature})[{node.threshold}]")
                self.print_tree(node.left)
                self.print_tree(node.right)
            else:
                print(f"{node.current_depth - 1}->{node.path}->{node.current_depth}:`Decision`({node.feature})[{node.threshold}]")
                self.print_tree(node.left)
                self.print_tree(node.right)
                
                
    
    def inorder_traversal(self, node = None):
        
        if not node:
            node = self.root
        
        if node.left:
            self.inorder_traversal(node.left)
        print(node)
        if node.right:
            self.inorder_traversal(node.right)
            
            
            
    def preorder_traversal(self, node = None):
        
        if not node:
            node = self.root
        
        print(node)
        if node.left:
            self.preorder_traversal(node.left)
   
        if node.right:
            self.preorder_traversal(node.right)
            
            
    def postorder_traversal(self, node = None):
        
        if not node:
            node = self.root
  
        if node.left:
            self.postorder_traversal(node.left)
   
        if node.right:
            self.postorder_traversal(node.right)
        print(node)

        
        


        

In [15]:
# main
from sklearn.model_selection import train_test_split

X = iris_df.iloc[:,:-1]
y = iris_df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 41)

tree = DecisionTreeClassifier(min_samples_split = 10, max_depth = 5,min_info_gain = .1)
tree.fit(X_train,y_train)

In [16]:
tree.preorder_traversal()

||🤔petal length (cm) <= 1.9    ⬆0.93
|----|🍃Left: 0
|----|🤔Right:petal width (cm) <= 1.5    ⬆0.77
|--------|🤔Left:petal length (cm) <= 4.9    ⬆0.18
|------------|🍃Left: 1
|------------|🍃Right: 2
|--------|🤔Right:petal length (cm) <= 5.0    ⬆0.12
|------------|🍃Left: 2
|------------|🍃Right: 2


In [20]:
def validate(node):
    print('node entropy:',node.entropy_node)
    print('left entropy:',node.entropy_left_child)
    print('right entropy:',node.entropy_right_child)
    print('left length:',node.length_left_child)
    print('right lenght:',node.length_right_child)
    print('node lenght:',node.length_node)
    print('left weight:',node.w_left)
    print('right weight:',node.w_right)
    print('children entropy:',node.entropy_children)
    print('information gain:',node.max_info_gain)
    print('node value:',node.value)

validate(tree.root)

node entropy: 1.5846619079379884
left entropy: 0.0
right entropy: 0.9998844148717589
left length: 41
right lenght: 79
node lenght: 120
left weight: 0.3416666666666667
right weight: 0.6583333333333333
children entropy: 0.6582572397905746
information gain: 0.9264046681474138
node value: None


In [21]:
validate(tree.root.left)

node entropy: 0.0
left entropy: 0.0
right entropy: 0.0
left length: 30
right lenght: 11
node lenght: 41
left weight: 0.7317073170731707
right weight: 0.2682926829268293
children entropy: 0.0
information gain: 0.0
node value: 0


In [29]:
validate(tree.root.right)

node entropy: 0.9998844148717589
left entropy: 0.17556502585750278
right entropy: 0.2811937964320427
left length: 38
right lenght: 41
node lenght: 79
left weight: 0.4810126582278481
right weight: 0.5189873417721519
children entropy: 0.23038502071264375
information gain: 0.7694993941591152
node value: None


In [30]:
validate(tree.root.right.left)

node entropy: 0.17556502585750278
left entropy: 0.0
right entropy: 0.0
left length: 37
right lenght: 1
node lenght: 38
left weight: 0.9736842105263158
right weight: 0.02631578947368421
children entropy: 0.0
information gain: 0.17556502585750278
node value: None
